<a href="https://colab.research.google.com/github/subramanya4shenoy/MachineLearningNbs/blob/main/Linear_Regeression_Problem-1/LR_Forecasting_Mini_Course_Sales(s2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem statement** ⚔

**Dataset Description**

For this challenge, you will be predicting a full year worth of sales for various fictitious learning modules from different fictitious Kaggle-branded stores in different (real!) countries. This dataset is completely synthetic, but contains many effects you see in real-world data, e.g., weekend and holiday effect, seasonality, etc. You are given the task of predicting sales during for year 2022.

Good luck!

In [1]:
"""
Basic setup for integrating Kaggle
Make sure the kaggle.json file is available and uploaded in session
"""
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
"""
Path to the dataset from kaggle
"""
!kaggle competitions download -c playground-series-s3e19

  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 128MB/s]


In [3]:
"""
unzipping the files and removing the zip
"""
!unzip \*.zip && rm *.zip

Archive:  playground-series-s3e19.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               




---



---





**Task**: To predict the sales during the year 2022

**Steps:**
  1. 👁 Look at the data and learn about the data.
  2. 🎯 Decide on which model to choose.
  3. 🦖 Do Exploratory Data analysis.
  4. 🧹 Prepare the data

In [89]:
"""
Importing packages for reading data
"""
import pandas as pd

salesdf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
submissiondf = pd.read_csv('sample_submission.csv')

In [90]:
def get_insights(df):
  print("\n\n===========dataframe==========================")
  print(df.head(3))
  print("\n================================================")

  print("\n\n===========dataframe size=====================")
  print(df.shape)
  print("\n================================================")

  print("\n\n===========dataframe column names=============")
  print(df.columns)
  print("\n================================================")

  print("\n\n===========dataframe Unique value=============")
  print(df.nunique())
  print("\n================================================")

  print("\n\n===========dataframe data types===============")
  print(df.info())
  print("\n================================================")

  print("\n\n===========dataframe descriptions=============")
  print(df.describe())
  print("\n================================================")
  print("==================================================")


In [91]:
"""
Understanding the data
"""
get_insights(salesdf)
# get_insights(testdf)
# get_insights(submissiondf)



===========dataframe==========================
   id        date    country         store  \
0   0  2017-01-01  Argentina  Kaggle Learn   
1   1  2017-01-01  Argentina  Kaggle Learn   
2   2  2017-01-01  Argentina  Kaggle Learn   

                                          product  num_sold  
0               Using LLMs to Improve Your Coding        63  
1                   Using LLMs to Train More LLMs        66  
2  Using LLMs to Win Friends and Influence People         9  



===========dataframe size=====================
(136950, 6)



===========dataframe column names=============
Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')



===========dataframe Unique value=============
id          136950
date          1826
country          5
store            3
product          5
num_sold      1028
dtype: int64



===========dataframe data types===============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 

### **Insights from data (EDA)**
1. We have training set of 6 columns and  136950 rows.
  *   2 int64 columns id and num_sold.
  *   Our dependant variable is num_sold. We need to predict num_sold
  *   We have date columns which need to be converted to date time
  *   We see countr, store and product columns which are categorical data. we need to convert them into numerical representation.
  *   We would be trying one-hot encoding for product columns and store columns as they have very small uniq values.


2. For test set we have similar data but num_sold column is missing (we will be predicting that)

3. For submission set we have id and num sold column. we will be filling the data, converting to csv and submitting.

   




In [129]:
"""
Visualizing the data and relationship with num_sold
"""
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

def visualize(tempDf):
  df = tempDf.copy()

  df['date'] = pd.to_datetime(df['date'])
  monthly_data = df.groupby(pd.Grouper(key='date', freq='D')).sum()
  plt.figure(figsize=(20, 6))

  # Line Plot with month-wise X-axis
  sns.lineplot(x=monthly_data.index, y='num_sold', data=monthly_data)

  # Format the X-axis to show months only
  ax = plt.gca()
  ax.xaxis.set_major_locator(mdates.MonthLocator())  # Set tick positions to be at the start of each month
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  # Format the date as 'Month Year' (e.g., 'Jan 2023')

  plt.title('Line Plot')
  plt.xlabel('Date')
  plt.ylabel('sales')
  plt.xticks(rotation=45)  # Rotate X-axis labels for better visibility
  plt.show()

  """
  Comparing relation ship with country and num_sold
  """
  # Line Plot
  sns.lineplot(x='country', y='num_sold', data=df)
  plt.title('Line Plot')
  plt.xlabel('date')
  plt.ylabel('num_sold')
  plt.show()


  """
  Comparing relation ship with product and num_sold
  """
  # Line Plot
  sns.lineplot(x='product', y='num_sold', data=df)
  plt.title('Line Plot')
  plt.xlabel('date')
  plt.ylabel('num_sold')
  plt.xticks(rotation=90)
  plt.show()

  """
  Comparing relation ship with store and num_sold
  """
  # Line Plot
  sns.lineplot(x='store', y='num_sold', data=df)
  plt.title('Line Plot')
  plt.xlabel('store')
  plt.ylabel('num_sold')
  plt.xticks(rotation=90)
  plt.show()

### Observation from each independant columns and target column
1. num_sold vs date range we see a outliers in 2020. we need to remove the outliers from the df before training. Also, creating a column for holidays and weekends.
2. We see some very low values in country (argentina)
3. We see some very low values for one of the course.
4. we will try to combine both and see if any columns can be removed.
5. We do see very low sales on the Store as well
6. Converting all the categorical columns (country, store, product) into numeric values.
7. creating permutation and combination of all the 3 categorical column and doing one-hot-encoding on them

In [143]:
"""
Part of EDA updating date column,
converting categorical data into numbers
one-hot-encoding

we will be doing the same for both salesdf (training + test set) and testdf
so creating methods to each operations first.
"""

"""
date-column
1. method to converting all date column to datetime
2. given " This dataset is completely synthetic, but contains many effects you see in real-world data,
    e.g., weekend and holiday effect, seasonality, etc"
3. we might need to introduce more columns based on holidays, weekends.
4. We import holiday package to get seasonal holiday of any country
"""

import holidays
import numpy as np

# Create a dictionary to store holiday classes for different countries
holiday_classes = {}

def generate_holidays(df):
  for country in df['country'].unique():
    try:
        holiday_classes[country] = getattr(holidays, country)()
    except AttributeError:
        holiday_classes[country] = None


def prepare_date_column(tempdf):
  df = tempdf.copy()

  # Check if date is weekend then True else False. [5 = saturday, 6 = sunday]
  df['Is_Weekend'] = df['date'].dt.dayofweek.isin([5, 6])

  # Converting True, False into 1, 0
  df['Is_Weekend'] = df['Is_Weekend'].astype(int)

  # Checking for country specific seasonal holidays
  df['Is_Seasonal_Holiday'] = 0

  # generating special holidays for countries
  generate_holidays(df)

  # updating the seasonal holiday column
  for index, row in df.iterrows():
    country = row['country']
    if country in holiday_classes and holiday_classes[country] is not None and row['date'] in holiday_classes[country]:
        df.at[index, 'Is_Seasonal_Holiday'] = 1

  return df

In [291]:
"""
Making the changes and visualizing again
1. removing date outlier
"""

salesdf['date'] = pd.to_datetime(salesdf['date'])

start_date = pd.to_datetime('2020-02-01') # starting of outliers
end_date = pd.to_datetime('2021-01-01') #ending of outliers

filtered_data = salesdf[~((salesdf['date'] >= start_date) & (salesdf['date'] <= end_date))]
mean_sales = filtered_data['num_sold'].mean()

# identifyin the outliers and removing
def removing_date_outliers(tempdf):
  df = tempdf.copy()
  df['date'] = pd.to_datetime(df['date'])

  # Use boolean indexing to filter the rows within the date range
  rows_to_update = (df['date'] >= start_date) & (df['date'] <= end_date)

  # Update the 'num_sold' values within the date range with the mean value
  df.loc[rows_to_update, 'num_sold'] = mean_sales
  return df

In [145]:
"""
2. Applying holiday columns, weekend columns
"""
def transform_date(tempdf):
  df = tempdf.copy()
  df = removing_date_outliers(df)
  df = prepare_date_column(df)
  return df

In [146]:
"""
3. Converting all categorical column into numbers
"""
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

def prepare_columns(tempdf):
  df = tempdf.copy()
  df['country'] = label_encoder.fit_transform(df['country'])
  df['store'] = label_encoder.fit_transform(df['store'])
  df['product'] = label_encoder.fit_transform(df['product'])
  return df

In [262]:
"""
  Country has uniq values so we will be converting into numbers via skLearn label encoders
  Store too has the same we will be number coding them via label_encoders
  Product is also limitted and will be number coding them and doing a one-hot encoding to the column via
  get_dummies
"""
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

def add_derived_feature_columns(tempdf):
  df = tempdf.copy()
  # new columns with country-store-product
  # Remove newline characters using strip() method
  # Remove newline characters using apply() and lambda function
  df['country-str'] = df['country'].apply(lambda x: str(x).strip())
  df['store-str'] = df['store'].apply(lambda x: str(x).strip())
  df['product-str'] = df['product'].apply(lambda x: str(x).strip())

  # Concatenate the columns without \n
  df['c-s-p'] = df['country-str'] + "-" + df["store-str"] + "-" + df["product-str"]

  # we will be converting this column into one-hot encoding and then use dummies
  # We can remove country, store, product columns then
  df['c-s-p'] = label_encoder.fit_transform(df['c-s-p'])

  columns_to_remove = ['country-str', 'store-str', 'product-str']
  df = df.drop(columns=columns_to_remove, axis=0)

  return df

In [263]:
def prepare_data(tempdf):
  df = tempdf.copy()
  df = transform_date(df)
  df = prepare_columns(df)
  df = add_derived_feature_columns(df)
  return df

In [264]:
def remove_high_vif_ols_column(tempdf):
  df = tempdf.copy()
  columns_to_remove = ['country', 'store', 'product', 'id']
  df = df.drop(columns=columns_to_remove, axis=0)
  return df

In [352]:
"""
Now we are able to complete our EDA and preprocessing data
Lets split the data for training and testing
"""
from sklearn.model_selection import train_test_split

id_values_train = salesdf['id'].values

sales_train_df = prepare_data(salesdf)

# # Below operation is performed post VIF and OLS observation
sales_train_df = remove_high_vif_ols_column(sales_train_df)
# # Above operation is performed post VIF and OLS observation

indices_to_remove = np.where(sales_train_df.columns == 'num_sold')[0]
category_columns = np.delete(sales_train_df.columns, indices_to_remove)

X = sales_train_df[category_columns]
y = sales_train_df['num_sold']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train['date'] = X_train['date'].astype(np.int64)
X_val['date'] = X_val['date'].astype(np.int64)

In [353]:
"""
OLS Model creation and VIF fo analyzing the columns
"""

import statsmodels.api as sm

X_train = sm.add_constant(X_train)
# Creating a first fitted model
lm_1 = sm.OLS(y_train, X_train).fit()


In [354]:
print(lm_1.summary())

                            OLS Regression Results                            
Dep. Variable:               num_sold   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 29 Jul 2023   Prob (F-statistic):                nan
Time:                        15:30:01   Log-Likelihood:            -7.1736e+05
No. Observations:              109560   AIC:                         1.435e+06
Df Residuals:                  109559   BIC:                         1.435e+06
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                6.945e-35   2

In [368]:
X_test_with_const = sm.add_constant(X_val)
print(X_test_with_const)
# Make predictions using the fitted model
y_val_pred = lm_1.predict(X_test_with_const)
y_val_pred



        const                 date  Is_Weekend  Is_Seasonal_Holiday  c-s-p
135486    1.0  1639267200000000000           1                    0     41
112235    1.0  1612483200000000000           0                    0     40
73585     1.0  1567987200000000000           0                    0      0
132039    1.0  1635292800000000000           0                    0     44
7090      1.0  1491350400000000000           0                    0     30
...       ...                  ...         ...                  ...    ...
3807      1.0  1487548800000000000           0                    0     47
43333     1.0  1533081600000000000           0                    0     48
6783      1.0  1491004800000000000           1                    0     38
55899     1.0  1547596800000000000           0                    0     29
25512     1.0  1512604800000000000           0                    0      2

[27390 rows x 5 columns]


135486    177.855983
112235    174.949992
73585     170.122299
132039    177.424771
7090      161.807417
             ...    
3807      161.394954
43333     166.335137
6783      161.769920
55899     167.909996
25512     164.113461
Length: 27390, dtype: float64

In [337]:
"""
test data preparing
"""

test_id_values = testdf['id'].values
sales_test_df = prepare_data(testdf)
sales_test_df = remove_high_vif_ols_column(sales_test_df)
columns_to_remove = ['num_sold']
sales_test_df = sales_test_df.drop(columns=columns_to_remove, axis=0)
sales_test_df['date'] = sales_test_df['date'].astype(np.int64)

In [338]:
"""
testing out test data
"""
X_test_with_const = sm.add_constant(sales_test_df)

# Check the number of features in X_test_with_const
print(X_test_with_const.shape[1])

# Make sure the number of features in X_test_with_const is the same as in the training data

# Make predictions using the fitted model
y_test_pred = lm_1.predict(X_test_with_const)

# Print or use the predictions as needed
print(y_test_pred)
X_test_with_const

5
0        178.043466
1        178.043466
2        178.043466
3        178.043466
4        178.043466
            ...    
27370    181.455661
27371    181.455661
27372    181.455661
27373    181.455661
27374    181.455661
Length: 27375, dtype: float64


,const,date,Is_Weekend,Is_Seasonal_Holiday,c-s-p
0,1.0,1640995200000000000,1,1,5
1,1.0,1640995200000000000,1,1,6
2,1.0,1640995200000000000,1,1,7
3,1.0,1640995200000000000,1,1,8
4,1.0,1640995200000000000,1,1,9
...,...,...,...,...,...
27370,1.0,1672444800000000000,1,0,60
27371,1.0,1672444800000000000,1,0,61
27372,1.0,1672444800000000000,1,0,62
27373,1.0,1672444800000000000,1,0,63


In [369]:
"""
Preparing the submission data
"""
import matplotlib.pyplot as plt

submissiondf = pd.DataFrame({
    'id': test_id_values,
    'num_sold': y_test_pred
})


In [ ]:

visualize(submissiondf)
submissiondf.to_csv('submission.csv', index=False)